# Find the function y of x

In this version, I just want to support these operations/symbols:

- Addition
- Subtraction
- Multiplication
- Division
- Exponentiation
- Parenthesis
- Input (x)
- Number

This does some really naive, random evolution. This is baseline; in future I want to try MCTS & intelligent evolution

In [542]:
import random

class Component:   
    child_count = 2
    
    def __init__(self, children=None):
        if(children is None):
            self.fill()
        elif(len(children) == self.child_count):
            self.children = children
            pass
        else:
            raise Exception('Initialized children must have length == child_count or be None')
    
    def compute(self, x):
        pass
    
    def fill(self):
        self.children = list([Number(1) for i in range(self.child_count)])
    
    def mutate(self, prob, decay):
        for index, child in enumerate(self.children):
            should_mutate = random.random() < prob
            
            if(should_mutate):
                options = [
                    Number(random.random()),
                    Input(),
                    Addition(),
                    Subtraction(),
                    Multiplication(),
                    Division(),
                    Exponentiation()
                ]
                choice = random.choice(options)
                self.children[index] = choice
                
            child.mutate(prob * decay, decay)

class Container(Component):
    child_count = 1
    
    def compute(self, x):
        return self.children[0].compute(x) 
                
class Exponentiation(Component):
    child_count = 2
    
    def compute(self, x):
        computed = [child.compute(x) for child in self.children]
        return Number(computed[0].value ** computed[1].value)    

class Multiplication(Component):
    child_count = 2
    
    def compute(self, x):
        computed = [child.compute(x) for child in self.children]
        return Number(computed[0].value * computed[1].value)
    
class Division(Component):
    child_count = 2
    
    def compute(self, x):
        computed = [child.compute(x) for child in self.children]
        return Number(computed[0].value / (computed[1].value if computed[1].value != 0 else 0.001))
    
class Subtraction(Component):
    child_count = 2
    
    def compute(self, x):
        computed = [child.compute(x) for child in self.children]
        return Number(computed[0].value - computed[1].value)
    
class Addition(Component):
    child_count = 2
    
    def compute(self, x):
        computed = [child.compute(x) for child in self.children]
        return Number(computed[0].value + computed[1].value)
    
class Input(Component):
    def __init__(self):
        pass
    
    def mutate(self, prob, decay):
        pass
    
    def compute(self, x):
        return Number(x)

class Number(Component):
    def __init__(self, value):
        self.value = value
        
    def mutate(self, prob, decay):
        pass
        
    # this can't compute further
    def compute(self, x):
        return self
    
    def __add__(self, other):
        print(self.value, other.value)
        return Number(self.value + other.value)
    
func = Container()

In [544]:
from copy import deepcopy

def loss(func):
    X = [1, 2, 3, 4]
    Y = [2, 4, 6, 8]
    
    error = []
    
    for index, x in enumerate(X):
        error.append(abs(Y[index] - func.compute(x).value))
        
    return sum(error) / len(error)

def evolve(func, loss):
    offspring_count = 250
    generations = 100
    mutation_prob = 0.6
    mutation_decay = 1.5
    epsilon = 0.05
    
    for generation in range(generations):
        best_func = func
        best_loss = loss(func)

        for offspring in range(offspring_count):
            new_func = deepcopy(func)
            new_func.mutate(mutation_prob, mutation_decay)
            new_func_loss = loss(new_func)
            
            if(new_func_loss == 0):
                return new_func
        
            if(random.random() < epsilon or new_func_loss <= best_loss):
                best_func = new_func
                best_loss = new_func_loss
                
        func = best_func
        
        print('GEN', generation, 'loss:', best_loss)
        
    return func
    
print('before', loss(func))
func = evolve(func, loss)
print('after', loss(func))

before 2.5
GEN 0 loss: 2.5
GEN 1 loss: 2.5
GEN 2 loss: 2.5
GEN 3 loss: 2.5
GEN 4 loss: 2.5
GEN 5 loss: 2.5
GEN 6 loss: 2.5
GEN 7 loss: 2.5
GEN 8 loss: 2.5
GEN 9 loss: 2.5
GEN 10 loss: 2.5
GEN 11 loss: 2.5
GEN 12 loss: 2.5
GEN 13 loss: 2.5
GEN 14 loss: 2.5
GEN 15 loss: 2.5
GEN 16 loss: 2.5
GEN 17 loss: 2.5
GEN 18 loss: 2.5
GEN 19 loss: 2.5
GEN 20 loss: 2.5
GEN 21 loss: 2.5
GEN 22 loss: 2.5
GEN 23 loss: 2.5
GEN 24 loss: 2.5
GEN 25 loss: 2.5
GEN 26 loss: 2.5
GEN 27 loss: 2.5
GEN 28 loss: 2.5
GEN 29 loss: 3.0
after 0.0
